In [ ]:
import torch

from learnedMethodForHologram import *
from learnedMethodForHologram.optics import bandLimitedAngularSpectrumMethod as BLASM_v1
from learnedMethodForHologram.bandlimited_angular_spectrum_approach import (
    bandLimitedAngularSpectrumMethod as BLASM_v2,
)

# about the data loader

In [ ]:
# convert the .exr data to torch tensors
data_processor.read_exr_in_multi_folders(
    "data\\test_192", channlesNum=3, height=192, width=192
)
data_processor.read_exr_in_multi_folders(
    "data\\test_384", channlesNum=3, height=384, width=384
)

In [ ]:
dataset = data_processor.data_loader(
    "data\\test_192\\amp.bin",
    "data\\test_192\\phs.bin",
    "data\\test_192\\img.bin",
    "data\\test_192\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
)

dataLoader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True, num_workers=0
)

In [ ]:
amp, phs, img, depth = next(iter(dataLoader))
utilities.multi_channel_plotter(utilities.tensor_normalizor_2D(amp), title=None, rgb_img=True)
utilities.multi_channel_plotter(utilities.tensor_normalizor_2D(phs), title=None, rgb_img=True)
utilities.multi_channel_plotter(utilities.tensor_normalizor_2D(img), title=None, rgb_img=True)
utilities.multi_channel_plotter(utilities.tensor_normalizor_2D(depth), title=None, rgb_img=True)

In [ ]:
amp, phs, img, depth = next(iter(dataLoader))
depth_np = depth.squeeze().numpy()
print(depth_np.max(), depth_np.min())

In [ ]:
import torch
from learnedMethodForHologram import *

dataset = data_processor.data_loader_A_B(
    path_A = "data\\MIT_CGH_192_bin\\train\\img.bin",
    path_B = "data\\MIT_CGH_192_bin\\train\\depth.bin",
    samplesNum=3800,
    channlesNum=3,
    height=192,
    width=192,
    padding=False,
    cuda=True,
)

dataLoader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)

%timeit img_depth_tensor = next(iter(dataLoader))
img_depth_tensor = next(iter(dataLoader))

print(f"the max value of the img is {img_depth_tensor[:,0].max()}")   


## data_loader_for_percepetual_loss

In [ ]:
import torch
from learnedMethodForHologram.data_processor import data_loader_for_percepetual_loss
from learnedMethodForHologram import utilities

# "data\\test_192\\amp.bin",
# "data\\test_192\\phs.bin",
# "data\\test_192\\img.bin",
# "data\\test_192\\depth.bin",

dataset = data_loader_for_percepetual_loss(
    path_img="data\\test_192\\img.bin",
    path_phs="data\\test_192\\phs.bin",
    path_depth="data\\test_192\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=False,
)

dataLoader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True, num_workers=0
)

amp_phs_depth_tensor = next(iter(dataLoader))

print(amp_phs_depth_tensor[:, :3].shape)

utilities.multi_sample_plotter(
    utilities.tensor_normalizor_2D(amp_phs_depth_tensor[:, :3]),
    titles=None,
    rgb_img=True,
)

utilities.multi_sample_plotter(
    utilities.tensor_normalizor_2D(amp_phs_depth_tensor[:, 3:-1]),
    titles=None,
    rgb_img=True,
)

utilities.multi_channel_plotter(
    utilities.tensor_normalizor_2D(amp_phs_depth_tensor[:, -1]),
    title=None,
    color=1,
)

# about band_limited_angular_spectrum_method(V1)

In [ ]:
torch.cuda.empty_cache()

distances = torch.tensor(
    [0.0, 2.5e-2, 5.0e-3, 7.5e-3, 1.0e-2, 5.0e-2, 10.0e-2, 20.0e-3]
)
propagator = BLASM_v1(distances=distances)
%timeit propagator.band_limited_angular_spectrum_multichannels()

del(propagator)
torch.cuda.empty_cache()

In [ ]:
# clear the cache
torch.cuda.empty_cache()

distances = torch.linspace(0, 2.5e-3, 3)
propagator = BLASM_v1(distances=distances,band_limit=True)
g_z_complex = propagator.band_limited_angular_spectrum_multichannels()
intensity = utilities.intensity_calculator(g_z_complex, True)
utilities.multi_sample_plotter(utilities.tensor_normalizor_2D(intensity), titles=None, rgb_img=True)

# about band_limited_angular_spectrum_method(V2)

In [ ]:
from tests import *
test_bandlimited_agular_spectrum_approach.test()

## dataset_192

In [ ]:
import torch
from learnedMethodForHologram import *

dataset = data_processor.data_loader(
    "data\\MIT_CGH_192_bin\\train\\amp.bin",
    "data\\MIT_CGH_192_bin\\train\\phs.bin",
    "data\\MIT_CGH_192_bin\\train\\img.bin",
    "data\\MIT_CGH_192_bin\\train\\depth.bin",
    samplesNum=3800,
    channlesNum=3,
    height=192,
    width=192,
)

dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)
sampleX = next(iter(dataLoader))

In [ ]:
amplitude_tensor = sampleX[0]
phase_tensor = sampleX[1]
distances = torch.linspace(-2e-4, 2e-4, 7)
# spacial_frequency_filter = (
#     utilities.generate_custom_frequency_mask(
#         sample_row_num=2400,
#         sample_col_num=4094,
#         x=800,
#         y=int(800 * 4094 / 2400),
#     )
# )
propagator = bandlimited_angular_spectrum_approach.bandLimitedAngularSpectrumMethod(
    sample_row_num=192,
    sample_col_num=192,
    pixel_pitch=3.74e-6,
    wave_length=torch.tensor([639e-9, 515e-9, 473e-9]),
    band_limit=False,
    cuda=False,
)
intensities = propagator(
    amplitute_tensor=amplitude_tensor,
    phase_tensor=phase_tensor,
    distances=distances,
    # spacial_frequency_filter=spacial_frequency_filter,
)
normalized_intensities = utilities.tensor_normalizor_2D(intensities)

utilities.multi_sample_plotter(
    normalized_intensities,
    titles=None,
    rgb_img=True,
    save_dir="output\\test_output\\blasm_v2_dataset_192",
    color=0,
)

## dataset_384

In [ ]:
import torch
from learnedMethodForHologram import *

dataset = data_processor.data_loader(
    "data\\MIT_CGH_384_bin\\test\\amp.bin",
    "data\\MIT_CGH_384_bin\\test\\phs.bin",
    "data\\MIT_CGH_384_bin\\test\\img.bin",
    "data\\MIT_CGH_384_bin\\test\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=384,
    width=384,
)

dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)
sampleX = next(iter(dataLoader))

In [ ]:
amplitude_tensor = sampleX[0]
phase_tensor = sampleX[1]
distances = torch.linspace(-0.15e-4, 10e-4, 7)
# spacial_frequency_filter = (
#     utilities.generate_custom_frequency_mask(
#         sample_row_num=2400,
#         sample_col_num=4094,
#         x=800,
#         y=int(800 * 4094 / 2400),
#     )
# )
propagator = bandlimited_angular_spectrum_approach.bandLimitedAngularSpectrumMethod(
    sample_row_num=384,
    sample_col_num=384,
    pixel_pitch=8e-6,
    wave_length=torch.tensor([638e-9, 520e-9, 450e-9]),
    band_limit=False,
    cuda=False,
)
intensities = propagator(
    amplitute_tensor=amplitude_tensor,
    phase_tensor=phase_tensor,
    distances=distances,
    # spacial_frequency_filter=spacial_frequency_filter,
)
normalized_intensities = utilities.tensor_normalizor_2D(intensities)

utilities.multi_sample_plotter(
    normalized_intensities,
    titles=None,
    rgb_img=True,
    save_dir="output\\test_output\\blasm_v2_dataset_384",
    color=0,
)

# about BLASM_v2 and v3

In [ ]:
from learnedMethodForHologram.bandlimited_angular_spectrum_approach import (
    bandLimitedAngularSpectrumMethod as BLASM_v2,
)
from learnedMethodForHologram.bandlimited_angular_spectrum_approach import (
    bandLimitedAngularSpectrumMethod_for_single_fixed_distance as BLASM_v3,
)

propagator_v2 = BLASM_v2(
    sample_row_num=192,
    sample_col_num=192,
    pixel_pitch=3.74e-6,
    wave_length=torch.tensor([639e-9, 515e-9, 473e-9]),
    band_limit=False,
    cuda=False,
)

w_grid_v2 = propagator_v2.generate_w_grid()
transfer_function_v2 = propagator_v2.generate_transfer_function(distances=torch.tensor([2.5e-3]))
print(transfer_function_v2.shape)

propagator_v3 = BLASM_v3(
    sample_row_num=192,
    sample_col_num=192,
    pixel_pitch=3.74e-6,
    wave_length=torch.tensor([639e-9, 515e-9, 473e-9]),
    band_limit=False,
    cuda=False,
    distance=torch.tensor([2.5e-3]),
)

w_grid_v3 = propagator_v3.generate_w_grid()
transfer_function_v3 = propagator_v3.generate_transfer_function()
print(transfer_function_v3.shape)

assert torch.allclose(w_grid_v2, w_grid_v3)
assert torch.allclose(transfer_function_v2, transfer_function_v3)

## dataset_192 with padding

In [ ]:
import torch
from learnedMethodForHologram import *

dataset = data_processor.data_loader(
    "data\\MIT_CGH_192_bin\\train\\amp.bin",
    "data\\MIT_CGH_192_bin\\train\\phs.bin",
    "data\\MIT_CGH_192_bin\\train\\img.bin",
    "data\\MIT_CGH_192_bin\\train\\depth.bin",
    samplesNum=3800,
    channlesNum=3,
    height=192,
    width=192,
    padding=True, # padding the input data to 256x256
    cuda=False,
)

dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)
sampleX = next(iter(dataLoader))

print(sampleX[0].shape)

In [ ]:
amplitude_tensor = sampleX[0]
phase_tensor = sampleX[1]
distances = torch.linspace(-1.5e-4, 1.5e-4, 7)
# spacial_frequency_filter = (
#     utilities.generate_custom_frequency_mask(
#         sample_row_num=2400,
#         sample_col_num=4094,
#         x=800,
#         y=int(800 * 4094 / 2400),
#     )
# )
propagator = bandlimited_angular_spectrum_approach.bandLimitedAngularSpectrumMethod(
    sample_row_num=512,
    sample_col_num=512,
    pixel_pitch=3.74e-6,
    wave_length=torch.tensor([639e-9, 515e-9, 473e-9]),
    band_limit=False,
    cuda=False,
)
intensities = propagator(
    amplitute_tensor=amplitude_tensor,
    phase_tensor=phase_tensor,
    distances=distances,
    # spacial_frequency_filter=spacial_frequency_filter,
)
normalized_intensities = utilities.tensor_normalizor_2D(intensities)

utilities.multi_sample_plotter(
    normalized_intensities,
    titles=None,
    rgb_img=True,
    save_dir="output\\test_output\\blasm_v2_dataset_192_padding_512",
    color=0,
)

# model training

In [ ]:
import torch
from learnedMethodForHologram import (
    bandlimited_angular_spectrum_approach,
    neural_network_components,
)

propagator = bandlimited_angular_spectrum_approach.bandLimitedAngularSpectrumMethod(
    sample_row_num=192,
    sample_col_num=192,
    pixel_pitch=3.74e-6,
    wave_length=torch.tensor([639e-9, 515e-9, 473e-9]),
    band_limit=False,
    cuda=False,
)

amps = torch.randn(4, 3, 192, 192)
phs = torch.randn(4, 3, 192, 192)

intensities = propagator(amps, phs, torch.tensor([0.0]))
print(f"intensities shape: {intensities.shape}")

amps = torch.randn(1, 3, 192, 192)
phs = torch.randn(1, 3, 192, 192)

intensities = propagator(amps, phs, torch.tensor([0.0]))
print(f"intensities shape: {intensities.shape}")

amps_phs = torch.randn(1, 6, 192, 192)

amp_phs = propagator.propagate_AP2AP(amps_phs, torch.tensor([0.0]))
print(f"propagate_AP2AP shape: {amp_phs.shape}")

amps = torch.randn(1, 3, 192, 192)
phs = torch.randn(1, 3, 192, 192)

intensities = propagator.propagate_P2I(phs, torch.tensor([0.0]))
print(f"propagate_P2I shape: {intensities.shape}")

amps_phs = torch.randn(4, 6, 192, 192)

amp_phs = propagator.propagate_AP2AP(amps_phs, torch.tensor([0.0]))
print(f"propagate_AP2AP shape: {amp_phs.shape}")

intensities = propagator.propagate_P2I(phs, torch.tensor([0.0]))
print(f"propagate_P2I shape: {intensities.shape}")

tensor1 = torch.randn(4, 4, 192, 192)
model = neural_network_components.UNet()
output = model(tensor1)
print(f"output shape of the UNet: {output.shape}")

tensor2 = torch.randn(4, 2, 192, 192)
model = neural_network_components.ResNet_POH()
output = model(tensor2)
print(f"output shape of the ResNet_POH: {output.shape}")

In [ ]:
import torch
from learnedMethodForHologram import (
    bandlimited_angular_spectrum_approach,
    neural_network_components,
)

propagator = bandlimited_angular_spectrum_approach.bandLimitedAngularSpectrumMethod_for_single_fixed_distance(
    sample_row_num=192,
    sample_col_num=192,
    pixel_pitch=3.74e-6,
    wave_length=torch.tensor([639e-9, 515e-9, 473e-9]),
    band_limit=False,
    cuda=False,
    distance=torch.tensor([0.0]),
)

amps = torch.randn(4, 3, 192, 192)
phs = torch.randn(4, 3, 192, 192)

intensities = propagator(amps, phs)
print(f"intensities shape: {intensities.shape}")

amps = torch.randn(1, 3, 192, 192)
phs = torch.randn(1, 3, 192, 192)

intensities = propagator(amps, phs)
print(f"intensities shape: {intensities.shape}")

amps_phs = torch.randn(1, 6, 192, 192)

amp_phs = propagator.propagate_AP2AP(amps_phs)
print(f"propagate_AP2AP shape: {amp_phs.shape}")

amps = torch.randn(1, 3, 192, 192)
phs = torch.randn(1, 3, 192, 192)

intensities = propagator.propagate_P2I(phs)
print(f"propagate_P2I shape: {intensities.shape}")

amps_phs = torch.randn(4, 6, 192, 192)

amp_phs = propagator.propagate_AP2AP(amps_phs)
print(f"propagate_AP2AP shape: {amp_phs.shape}")

intensities = propagator.propagate_P2I(phs)
print(f"propagate_P2I shape: {intensities.shape}")

tensor1 = torch.randn(4, 4, 192, 192)
model = neural_network_components.UNet()
output = model(tensor1)
print(f"output shape of the UNet: {output.shape}")

tensor2 = torch.randn(4, 2, 192, 192)
model = neural_network_components.ResNet_POH()
output = model(tensor2)
print(f"output shape of the ResNet_POH: {output.shape}")

In [ ]:
model = neural_network_components.UNet()
model.eval()
%timeit output = model(tensor1)

model = neural_network_components.ResNet_POH()
%timeit output = model(tensor2)

In [ ]:
import torch
from learnedMethodForHologram.data_processor import data_loader_A_B
from learnedMethodForHologram.watermelon_v1 import watermelon_v1

batch_num = 4
dataset = data_loader_A_B(
    path_A="data\\MIT_CGH_192_bin\\train\\img.bin",
    path_B="data\\MIT_CGH_192_bin\\train\\depth.bin",
    samplesNum=3800,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataLoader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_num,
    shuffle=True,
    num_workers=0,
)

batch = next(iter(dataLoader))
model = watermelon_v1(
    input_shape=batch.shape,
    propagation_distance=torch.tensor([2.5e-3]),
    cuda=True,
)
output = model(batch)

In [ ]:
import torch
from learnedMethodForHologram.data_processor import data_loader_A_B
from learnedMethodForHologram.watermelon_v1 import watermelon_v1

dataset_train = data_loader_A_B(
    path_A="data\\MIT_CGH_192_bin\\test\\img.bin",
    path_B="data\\MIT_CGH_192_bin\\test\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataLoader_dataset_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=1,
    shuffle=True,
    num_workers=0,
)

dataset_test = data_loader_A_B(
    path_A="data\\MIT_CGH_192_bin\\validate\\img.bin",
    path_B="data\\MIT_CGH_192_bin\\validate\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataLoader_dataset_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=True,
    num_workers=0,
)

model = watermelon_v1(
    input_shape=(1, 4, 192, 192),
    propagation_distance=torch.tensor([2.5e-3]),
    cuda=True,
)

model.train_model(
    train_iter=dataLoader_dataset_train,
    test_iter=dataLoader_dataset_test,
    num_epochs=10,
    lr=1e-2,
)

# save the model
torch.save(model.state_dict(), 'watermelon_v1.pth')

In [ ]:
import torch
from learnedMethodForHologram import utilities
from learnedMethodForHologram.data_processor import data_loader_A_B
from learnedMethodForHologram.watermelon_v1 import watermelon_v1


# load the model
model = watermelon_v1(
    input_shape=(1, 4, 192, 192),
    propagation_distance=torch.tensor([1e-3]),
    cuda=True,
)

model.load_state_dict(torch.load("output\\models\\watermelon_v2_trainedwith3800samples.pth"))
model.eval()

dataset_test = data_loader_A_B(
    path_A="data\\MIT_CGH_192_bin\\test\\img.bin",
    path_B="data\\MIT_CGH_192_bin\\test\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataLoader_dataset_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=True,
    num_workers=0,
)

batch = next(iter(dataLoader_dataset_test))
output = model(batch).to("cpu").detach()
print(f"output type: {type(output)}")
print(f"output shape: {output.shape}")
print(f"output max: {output.max()}")
print(f"output min: {output.min()}")
utilities.multi_sample_plotter(
    utilities.tensor_normalizor_2D(output),
    titles=["reconstructed intensity"],
    rgb_img=True,
)

# watermelon

## v1

In [ ]:
import torch
from learnedMethodForHologram.data_processor import data_loader_A_B
from learnedMethodForHologram.watermelon_v1 import watermelon_v1

dataset_train = data_loader_A_B(
    path_A="data/MIT_CGH_192_bin/test/img.bin",
    path_B="data/MIT_CGH_192_bin/test/depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

# dataset_train = data_loader_A_B(
#     path_A="data/MIT_CGH_192_bin/train/img.bin",
#     path_B="data/MIT_CGH_192_bin/train/depth.bin",
#     samplesNum=3800,
#     channlesNum=3,
#     height=192,
#     width=192,
#     cuda=True,
# )

dataLoader_dataset_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=4,
    shuffle=True,
    num_workers=0,
)

dataset_test = data_loader_A_B(
    path_A="data/MIT_CGH_192_bin/validate/img.bin",
    path_B="data/MIT_CGH_192_bin/validate/depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataLoader_dataset_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=True,
    num_workers=0,
)

model = watermelon_v1(
    input_shape=(4, 4, 192, 192),
    propagation_distance=torch.tensor([1e-3]),
    cuda=True,
)

model.train_model(
    train_iter=dataLoader_dataset_train,
    test_iter=dataLoader_dataset_test,
    num_epochs=10,
    # with v1,v2 lr = 1e-3
    lr=1e-4,
)

# save the model
# torch.save(model.state_dict(), 'output\\models\\watermelon_v2_trainedwith3800samples.pth')

In [ ]:
import torch
from learnedMethodForHologram.data_processor import data_loader_A_B
from learnedMethodForHologram.watermelon_v1 import watermelon_v1

# load the model
model = watermelon_v1(
    input_shape=(1, 4, 192, 192),
    propagation_distance=torch.tensor([1e-3]),
    cuda=True,
)

# model.load_state_dict(torch.load('output\\models\\watermelon_v2_trainedwith3800samples.pth'))
model.load_state_dict(torch.load('output\\models\\watermelon_v3_lr_1e-4_.pth'))

model.eval()

dataset_test = data_loader_A_B(
    path_A="data/MIT_CGH_192_bin/test/img.bin",
    path_B="data/MIT_CGH_192_bin/test/depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataLoader_dataset_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=True,
    num_workers=0,
)

In [ ]:
batch = next(iter(dataLoader_dataset_test)).detach()
amp_phs_z = model.part1(batch).detach()
amp_phs_0 = model.propagator.propagate_AP2AP(amp_phs_z).detach()
phs_0 = model.part2(amp_phs_0).detach()
intensity = model.propagator.propagate_P2I(phs_0).detach()

In [ ]:
from learnedMethodForHologram import utilities

input_phs_restruction = torch.cat((batch[:,0:3],phs_0,intensity),dim=0)

utilities.multi_sample_plotter(
    utilities.tensor_normalizor_2D(input_phs_restruction),
    titles = ["ground_truth_at_z1","POH_at_z0","reconstructed_intensity_at_z1"],
    rgb_img=True,
    # save_dir="output\\images\\1"
)

In [ ]:
from learnedMethodForHologram import utilities

input_phs_restruction = torch.cat((batch[:,0:3],phs_0,intensity),dim=0)

utilities.multi_sample_plotter(
    utilities.tensor_normalizor_2D(input_phs_restruction),
    titles = ["ground_truth_at_z1","POH_at_z0","reconstructed_intensity_at_z1"],
    rgb_img=True,
    # save_dir="output\\images\\1"
)

# v2

### perceptual loss

In [ ]:
import torch
from learnedMethodForHologram import perceptual_loss
from learnedMethodForHologram.data_processor import data_loader_for_percepetual_loss

# "data\\MIT_CGH_192_bin\\train\\depth.bin"

dataset_train = data_loader_for_percepetual_loss(
    path_img="data\\MIT_CGH_192_bin\\train\\img.bin",
    path_phs="data\\MIT_CGH_192_bin\\train\\phs.bin",
    path_depth="data\\MIT_CGH_192_bin\\train\\depth.bin",
    samplesNum=3800,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataset_validate = data_loader_for_percepetual_loss(
    path_img="data\\MIT_CGH_192_bin\\validate\\img.bin",
    path_phs="data\\MIT_CGH_192_bin\\validate\\phs.bin",
    path_depth="data\\MIT_CGH_192_bin\\validate\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

train_iter = torch.utils.data.DataLoader(
    dataset_train, batch_size=4, shuffle=True, num_workers=0
)

validate_iter = torch.utils.data.DataLoader(
    dataset_validate, batch_size=1, shuffle=True, num_workers=0
)

loss_model = perceptual_loss.perceptual_loss(
    input_shape=(2, 6, 192, 192),
    cuda=True,
)

loss_model.train_model(
    train_iter=train_iter,
    test_iter=validate_iter,
    num_epochs=30,
    lr=1e-4,
)

torch.save(loss_model.state_dict(), 'output\\models\\perceptual_loss_3800samples_lr4.pth')

In [ ]:
import torch
from learnedMethodForHologram import perceptual_loss
from learnedMethodForHologram.data_processor import data_loader_for_percepetual_loss
from learnedMethodForHologram import utilities

dataset_test = data_loader_for_percepetual_loss(
    path_img="data\\MIT_CGH_192_bin\\test\\img.bin",
    path_phs="data\\MIT_CGH_192_bin\\test\\phs.bin",
    path_depth="data\\MIT_CGH_192_bin\\test\\depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

model = perceptual_loss.perceptual_loss(
    input_shape=(2, 6, 192, 192),
    cuda=True,
)
model.load_state_dict(torch.load("output\\models\\perceptual_loss_3800samples_lr4.pth"))

n = 15

model.eval()
depth = model(dataset_test[n][:-1].unsqueeze(0)).detach()

utilities.multi_channel_plotter(
    depth,
    title="depth_hat",
    color=2,
)

print(f"depth max: {depth.max()}")
print(f"depth min: {depth.min()}")
print(f"depth mean: {depth.mean()}")

utilities.multi_channel_plotter(
    dataset_test[n][-1],
    title="Ground Truth",
    color=2,
)

print(f"depth max: {dataset_test[0][-1].max()}")
print(f"depth min: {dataset_test[0][-1].min()}")
print(f"depth mean: {dataset_test[0][-1].mean()}")

## v2 training

In [ ]:
import torch
from learnedMethodForHologram.data_processor import (
    data_loader_A_B as data_loader_img_depth,
)
from learnedMethodForHologram.watermelon_v2 import watermelon_v2 as watermelon

dataset_train = data_loader_img_depth(
    path_A="data/MIT_CGH_192_bin/test/img.bin",
    path_B="data/MIT_CGH_192_bin/test/depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataset_train = data_loader_img_depth(
    path_A="data/MIT_CGH_192_bin/train/img.bin",
    path_B="data/MIT_CGH_192_bin/train/depth.bin",
    samplesNum=3800,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataset_validate = data_loader_img_depth(
    path_A="data/MIT_CGH_192_bin/validate/img.bin",
    path_B="data/MIT_CGH_192_bin/validate/depth.bin",
    samplesNum=100,
    channlesNum=3,
    height=192,
    width=192,
    cuda=True,
)

dataLoader_dataset_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=4,
    shuffle=True,
    num_workers=0,
)


dataLoader_dataset_validate = torch.utils.data.DataLoader(
    dataset_validate,
    batch_size=1,
    shuffle=True,
    num_workers=0,
)


model = watermelon(
    input_shape=(4, 4, 192, 192),
    perceptual_model_path="output/models/perceptual_loss_3800samples_lr4.pth",
    propagation_distance=torch.tensor([1e-3]),
    cuda=True,
)

model.train_model(
    train_iter=dataLoader_dataset_train,
    test_iter=dataLoader_dataset_validate,
    num_epochs=10,
    lr=1e-3,
    hyperparameter_lambda = 1e-3,
)

# drafts

In [ ]:
sample_row_num = 2400
sample_col_num = 4094
x = sample_row_num // 3
y = x * sample_col_num // sample_row_num
x, y

In [ ]:
tensorX = torch.rand(3,5,256,256)
tensor_192 = tensorX[
    ...,
    256//2 - 192//2 : 256//2 + 192//2,
    256//2 - 192//2 : 256//2 + 192//2,
]
print(tensor_192.shape)

In [ ]:
import torch
from learnedMethodForHologram import utilities
tensorX = torch.rand(3,5,256,256)
%timeit utilities.cut_center_256_192(tensorX).shape

In [ ]:
import torch

# 创建一个张量并启用梯度计算
x = torch.tensor(
    [[1.0, 2.0, 3.0], [3.0, 4.0, 5.0], [4.0, 5.0, 6.0]], requires_grad=True
)

# 对张量进行裁剪，并将裁剪的部分分离出计算图
y = x[:1, :].detach()

# 进行一些计算
z = y.sum() + x.sum()

# 反向传播
z.backward()

# 查看梯度
print(y.grad)  # 只有没有被分离的部分会有梯度

In [ ]:
import torch

# 创建一个张量并启用梯度计算
x = torch.tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)

# 对张量进行裁剪，并将裁剪的部分分离出计算图
x = x[:1, :]
x.retain_grad()

# 进行一些计算
z = x.sum()

# 反向传播
z.backward()

# 查看梯度
print(x.grad)  # 只有没有被分离的部分会有梯度

In [ ]:
import torch

# 假设的行数和列数
samplingRowNum = 10
samplingColNum = 20

# 示例张量，假设形状为 (100, 3, 2, 10, 20)
amp_phs_tensor = torch.randn(100, 6, samplingRowNum, samplingColNum)

# 重新定义形状，并去除大小为1的维度
result_tensor = amp_phs_tensor.view(-1, 3, 2, samplingRowNum, samplingColNum)[
    :, :, 0
].squeeze()

print(result_tensor.shape)

In [ ]:
import torch

def custom_loss(output, target):
    # 计算每个像素的绝对差异
    loss = torch.mean(torch.abs(output - target))
    return loss

# 假设 batch_num 为 x，图像大小为 192x192，三通道
batch_num = 4
output = torch.randn(batch_num, 3, 192, 192)  # 模型输出
target = torch.randn(batch_num, 3, 192, 192)  # 目标图像

# 计算自定义损失
loss = custom_loss(output, target)
print(f'Custom Loss: {loss.item()}')


In [ ]:
x = torch.randn(4, 3, 192, 192)
y = x[:,:2]
print(y.shape)

In [ ]:
import numpy as np

# 生成示例复光场数据（这里只是一个示例，不是实际数据）
# 复光场数据通常是一个 4D 数组，包含振幅和相位信息
L = np.random.rand(100, 100, 100, 100) + 1j * np.random.rand(100, 100, 100, 100)

# 提取特定视角的信息
def extract_view(L, angle_u, angle_v):
    # 对复光场数据进行傅里叶变换
    L_ft = np.fft.fftshift(np.fft.fft2(L, axes=(0, 1)))
    
    # 选择特定角度的视角
    u_index = int((L.shape[0] / 2) + angle_u * (L.shape[0] / 2))
    v_index = int((L.shape[1] / 2) + angle_v * (L.shape[1] / 2))
    
    # 提取该视角下的图像信息
    view = L_ft[u_index, v_index, :, :]
    
    # 对提取的视角图像进行逆傅里叶变换
    view_image = np.fft.ifft2(np.fft.ifftshift(view))
    
    return np.abs(view_image)

import matplotlib.pyplot as plt

# 提取不同角度的视角图像
angle_u, angle_v = 0.1, 0.2  # 示例角度
view_image = extract_view(L, angle_u, angle_v)

# 显示视角图像（这里省略具体显示代码，使用 matplotlib 或其他图像显示库）
plt.imshow(view_image, cmap='gray')
plt.title(f'View at angles ({angle_u}, {angle_v})')
plt.show()

# 提取不同角度的视角图像
angle_u, angle_v = -0.1, -0.2  # 示例角度
view_image = extract_view(L, angle_u, angle_v)

# 显示视角图像（这里省略具体显示代码，使用 matplotlib 或其他图像显示库）
plt.imshow(view_image, cmap='gray')
plt.title(f'View at angles ({angle_u}, {angle_v})')
plt.show()


## some exploration in the frequency domain

In [ ]:
import torch
from learnedMethodForHologram.data_processor import data_loader_A_B

dataset_train = data_loader_A_B(
    path_A="data/MIT_CGH_192_bin/train/img.bin",
    path_B="data/MIT_CGH_192_bin/train/depth.bin",
    samplesNum=3800,
    channlesNum=3,
    height=192,
    width=192,
    cuda=False,
)

In [ ]:
from learnedMethodForHologram import utilities
import matplotlib.pyplot as plt

img_spatial_domain = torch.sqrt(dataset_train[0][0:3])
utilities.multi_channel_plotter(
    utilities.tensor_normalizor_2D(dataset_train[0][0:3]),
    title="image in spatial domain",
    # save_dir="output\images\lowpass_compare",
)

img_freq_domain = torch.fft.fftshift(torch.fft.fft2(img_spatial_domain), dim=(-1, -2))
img_freq_domain_amp = torch.abs(img_freq_domain)
img_freq_domain_phs = torch.angle(img_freq_domain)
img_freq_domain_amp_log = torch.log(img_freq_domain_amp)
utilities.multi_channel_plotter(
    utilities.tensor_normalizor_2D(img_freq_domain_amp_log),
    title="log amplitude in frequency domain(normalized)",
)
utilities.multi_channel_plotter(
    utilities.tensor_normalizor_2D(img_freq_domain_phs),
    title="phase in frequency domain",
)
print(f"the max value of the phs is {img_freq_domain_phs.max()}")
print(f"the min value of the phs is {img_freq_domain_phs.min()}")

# for i in range(3):
#     plt.imshow(img_freq_domain_amp_log[i].numpy())
#     plt.colorbar()
#     plt.title(f"channel {i}")
#     plt.show()

mask = torch.fft.fftshift(utilities.generate_circular_frequency_mask(192, 192, 60))
masked_img_freq_domain = img_freq_domain * mask
masked_img_freq_domain_amp = img_freq_domain_amp_log * mask

utilities.multi_channel_plotter(
    utilities.tensor_normalizor_2D(masked_img_freq_domain_amp),
    title="amplitude in frequency domain after filtering",
)

masked_img_spatial_domain = torch.fft.ifft2(
    torch.fft.ifftshift(masked_img_freq_domain, dim=(-1, -2))
)

masked_img_spatial_domain_amp = torch.abs(masked_img_spatial_domain)**2

utilities.multi_channel_plotter(
    utilities.tensor_normalizor_2D(masked_img_spatial_domain_amp),
    title="amplitude in frequency domain of ground truth",
    # save_dir="output\images\lowpass_compare",
)

masked_img_spatial_domain_img = torch.abs(masked_img_spatial_domain) ** 2

utilities.multi_channel_plotter(
    utilities.tensor_normalizor_2D(masked_img_spatial_domain_img),
    title="image in spatial domain after filtering",
    # save_dir="output\images\lowpass_compare",
)

In [ ]:
from learnedMethodForHologram import utilities
import matplotlib.pyplot as plt

import torch
mask = utilities.generate_circular_frequency_mask(192, 192, 60)
print(mask[mask == 1.0].shape)

mask = torch.fft.fftshift(mask)
plt.imshow(mask)
plt.colorbar()
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from learnedMethodForHologram import utilities


def generate_circle_frequency_mask(
    sample_row_num=192,
    sample_col_num=192,
    radius=60,
    decay_rate=None,
):
    shorter_edge = min(sample_row_num, sample_col_num)
    if radius > shorter_edge / 2:
        raise ValueError(
            f"The radius {radius} is larger than the half of the sample size {shorter_edge/2}"
        )

    # Create a grid of (u, v) coordinates
    u = torch.fft.fftfreq(sample_row_num).unsqueeze(-1)
    v = torch.fft.fftfreq(sample_col_num).unsqueeze(0)
    # enable the circle change to ellipse
    D = torch.sqrt(u**2 + v**2) * shorter_edge

    mask = torch.ones_like(D)
    if decay_rate is not None:
        # Create the circular low-pass filter with exponential decay around the radius
        mask[D > radius] = torch.exp(-decay_rate * (D[D > radius] - radius))
    else:
        mask[D > radius] = 0.0

    return mask


filter1 = generate_circle_frequency_mask()
filter2 = utilities.generate_circular_frequency_mask(192, 192, 60)
assert torch.allclose(filter1, filter2)

filter = torch.fft.fftshift(filter2)
plt.imshow(filter)
plt.colorbar()
plt.title("Filter kernel")
plt.show()

cut = filter[95, :]
plt.plot(cut)
plt.title("Filter kernel cut")
plt.show()

plt.plot(torch.log(cut))
plt.title("Filter kernel cut")
plt.show()

In [ ]:
n=12
torch.fft.fftfreq(n,1)*n


In [ ]:
torch.fft.fftfreq(100,1)